In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

import subprocess


In [2]:
temp = subprocess.getoutput("ls -lah keys/80_1024*/b*")


In [3]:
def splitter(test):
    tmp = test.split()
    tst = [tmp[4]]
    tst += tmp[8].split("/")[1].split("_")
    return tst

# splitter(test)

df = pd.DataFrame(
    [splitter(line) for line in temp.split("\n")], columns=["disk","bs","poly","dim","base_log","level"])
df["disk"] = df["disk"].apply(lambda x: int(x[:-1]))

df.head()

,disk,bs,poly,dim,base_log,level
0,193,80,1024,1,3,3
1,257,80,1024,1,3,4
2,321,80,1024,1,3,5
3,385,80,1024,1,3,6
4,449,80,1024,1,3,7


In [4]:
df.pivot_table(values="disk", index="level", columns="base_log")


base_log,3,4,5,6,7
level,,,,,
3,193,193,193,193,193
4,257,257,257,257,257
5,321,321,321,321,321
6,385,385,385,385,385
7,449,449,449,449,449
8,513,513,513,513,513
9,577,577,577,577,577


In [ ]:
temp = !cat boot_stats.txt

for base_log in range(3,8):
    for level in range(3,10):
        test = "keys/80_1024_1_%d_%d"%(base_log,level)
        if test not in [line.split()[0] for line in temp]:
            print("*"*80, "\n", test, "\n", "*"*80, "\n")
            !target/debug/test_boot $base_log $level 
            print("*"*80, "\n\n")
        

******************************************************************************** 
 keys/80_1024_1_3_3 
 ******************************************************************************** 

base_log: 3
level: 3
key path: keys/80_1024_1_3_3
loading LWE sk 0... 

x [4.0]

x* [4.000000000022188]
oooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo

v0 0.0000000000000000000000008271806125530277
n0 26
s0 38
loading BSK 00... 

Loss of precision during bootstrap: 6 bit(s) of precision lost over 3 bit(s) of message originally. Consider increasing the number of level and/or decreasing the log base.
keys/80_1024_1_3_3 NA 19 18493 2880
******************************************************************************** 


******************************************************************************** 
 keys/80_1024_1_3_4 
 ******************************************************************************** 

base_log: 3
level: 4
key path: keys/80_1024_1_3_4
loading LWE sk 0... 

x [4.0]

x* [4

In [9]:
def splitter2(test):
    tmp = test.replace("NA","0").split()
    return tmp[0].split("/")[1].split("_") + tmp[1:]

temp = !cat boot_stats.txt

[splitter2(line) for line in temp]
df = pd.DataFrame([splitter2(line) for line in temp], 
        columns=["bs","poly","dim","base_log","level","free", "sk_load", "bsk_load", "boot"])

df.head()
                                                     

,bs,poly,dim,base_log,level,free,sk_load,bsk_load,boot
0,80,1024,1,5,3,5,13,22613,2893
1,80,1024,1,6,4,14,13,24416,3779
2,80,1024,1,3,3,0,19,18493,2880
3,80,1024,1,3,4,0,22,23930,3735
4,80,1024,1,3,6,8,16,35600,5067


In [10]:
df.pivot_table(values="free", index="level", columns="base_log")


base_log,3,5,6
level,,,
3,0.0,5.0,NaN
4,0.0,NaN,14.0
6,8.0,NaN,NaN


In [11]:
df.pivot_table(values="sk_load", index="level", columns="base_log")


base_log,3,5,6
level,,,
3,19.0,13.0,NaN
4,22.0,NaN,13.0
6,16.0,NaN,NaN


In [12]:
df.pivot_table(values="bsk_load", index="level", columns="base_log")


base_log,3,5,6
level,,,
3,18493.0,22613.0,NaN
4,23930.0,NaN,24416.0
6,35600.0,NaN,NaN


In [13]:
df.pivot_table(values="boot", index="level", columns="base_log")


base_log,3,5,6
level,,,
3,2880.0,2893.0,NaN
4,3735.0,NaN,3779.0
6,5067.0,NaN,NaN
